In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import copy

In [2]:
# forward prop
## sigmoid activation function using pytorch
def sigmoid_activation(z):
    return 1 / (1 + torch.exp(-z))

def forward_prop(_data, _nodes, _weights, _biases):
    layer_vals = []
    a = _data
    for i in range(len(_nodes)-2): # feed thru hidden layers
        z = torch.mm(a, _weights[i]) + _biases[i]
        a = sigmoid_activation(z)
        layer_vals.append(a)
    z_out = torch.mm(a, _weights[-1]) + _biases[-1]
    output = sigmoid_activation(z_out)
    layer_vals.append(output)
#     print(output)
    return layer_vals

In [3]:
# loss computation
# loss = y - output
def calculate_loss(_layer_vals):
    return y - _layer_vals[-1]

In [4]:
# back propogation
## function to calculate the derivative of activation
def sigmoid_delta(x):
  return x * (1 - x)

In [5]:
def backprop_and_update(_layer_vals, _weights, _biases, _data):
    loss = calculate_loss(_layer_vals)
    deltas = [] # from first hidden to output
    ds = [] # from output to first hidden
    # compute derivative of error terms
    for a in _layer_vals:
        deltas.append(sigmoid_delta(a))
    loss_d = loss
    for d, w in zip(reversed(deltas), reversed(_weights)):
        dd = loss * d
        loss_d = torch.mm(d, w.t())
        ds.append(dd)
    learning_rate = 0.1
    _as = copy.deepcopy(_layer_vals)
    _as.insert(0,x)
    del _as[-1]
#     print("\n Layer vals \n", _layer_vals)
    new_weights = []
    new_biases = []
    print("lengths", len(ds), len(_weights), len(_as))
    for d, w, a, b in zip(ds, reversed(_weights), _as, reversed(_biases)):
        wt = torch.mm(a.t(),d) * learning_rate
        new_weights.insert(0, wt)
        
        bi = b + d.sum()*learning_rate
        new_biases.insert(0,bi)
    return new_weights,  new_biases   

In [6]:
def train(_data, _labels, _nodes, _epochs):
    #param initialization
    weights = []
    for i in range(len(_nodes)-1):
        w = torch.randn(_nodes[i], _nodes[i+1])
        weights.append(w)
    biases = []
    for i in range(len(nodes)-1):
        b = torch.randn(1, nodes[i+1])
        biases.append(b)
    
    print("WEIGHTS ARRAY:\n", weights)
    print("BIASES ARRAY:\n", biases)

    print("\n TRAINING \n")
    y_out = forward_prop(_data, _nodes, weights, biases)
    weights_2, biases_2 = backprop_and_update(y_out, weights, biases, _data)
    print("Final Weights: \n", weights_2)
    print("Final Biases: \n", biases_2)
    
   

In [7]:
## initialize tensor for inputs, and outputs 
x = torch.randn((1, 5))
y = torch.randn((1, 1))
nodes = [5, 3, 2, 1]
train(x, y, nodes, 10)

WEIGHTS ARRAY:
 [tensor([[-2.1934, -0.1694, -0.7658],
        [ 0.2024, -2.0058, -0.5924],
        [-1.1959,  0.6700,  0.7086],
        [ 0.7521, -0.2031, -1.0889],
        [-0.9813,  0.5011, -0.3183]]), tensor([[ 0.2683, -1.3406],
        [ 0.1952,  0.8453],
        [-1.0682,  0.5829]]), tensor([[ 0.3317],
        [-0.7129]])]
BIASES ARRAY:
 [tensor([[-0.6620,  0.1558,  0.9170]]), tensor([[-0.5897,  0.3151]]), tensor([[-2.1740]])]

 TRAINING 

lengths 3 3 3
Final Weights: 
 [tensor([[0.0001, 0.0004, 0.0011],
        [0.0001, 0.0004, 0.0012]]), tensor([[0.0029, 0.0030],
        [0.0003, 0.0003],
        [0.0013, 0.0014]]), tensor([[-0.0013],
        [ 0.0009],
        [-0.0018],
        [ 0.0002],
        [ 0.0006]])]
Final Biases: 
 [tensor([[-0.6570,  0.1608,  0.9221]]), tensor([[-0.5837,  0.3212]]), tensor([[-2.1728]])]
